# BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [21]:
!pip install langchain_google_genai

  Using cached langchain_google_genai-0.0.6-py3-none-any.whl (15 kB)
  Using cached google_generativeai-0.3.2-py3-none-any.whl (146 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl (598 kB)
  Using cached google_auth-2.27.0-py2.py3-none-any.whl (186 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 4.9 MB/s eta 0:00:00
  Using cached protobuf-4.25.2-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl (48 kB)
  Using cached googleapis_common_protos-1.62.0-py2.py3-none-any.whl (228 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl (84 kB)


In [7]:
!pip install --upgrade --quiet  duckduckgo-search

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [3]:
import os


## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [55]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [5]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", )
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [6]:
from langchain.globals import set_debug
set_debug(True)

In [7]:
llm = GoogleGenerativeAI(model='gemini-pro', )


In [8]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
# from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import DuckDuckGoSearchRun
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
# search = SerpAPIWrapper()
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [9]:
llm = OpenAI(temperature=0)
# llm = llm
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/home/none/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [10]:
llm_chain.get_prompts()

[]

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [11]:
OBJECTIVE = "Write a weather report for SF today"

In [12]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 4
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [13]:
baby_agi({"objective": OBJECTIVE})

/home/none/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[chain/start] [1:chain:BabyAGI] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today"
}

*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list
[chain/start] [1:chain:BabyAGI > 2:chain:AgentExecutor] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today",
  "context": "",
  "task": "Make a todo list"
}
[chain/start] [1:chain:BabyAGI > 2:chain:AgentExecutor > 3:chain:LLMChain] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today",
  "context": "",
  "task": "Make a todo list",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:BabyAGI > 2:chain:AgentExecutor > 3:chain:LLMChain > 4:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an AI who performs one task based on the following objective: Write a weather report for SF today. Take into account these previously completed tasks: .\n\nSear

/home/none/miniconda3/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


[tool/end] [1:chain:BabyAGI > 24:chain:AgentExecutor > 27:tool:Search > 28:tool:duckduckgo_search] [1.74s] Exiting Tool run with output:
"Today Hourly 10-Day Calendar History Wundermap access_time 2:13 AM PST on February 11, 2024 (GMT -8) | Updated 19 hours ago --° | 44° 54 °F like 54° Partly Cloudy N 0 Tomorrow's temperature is... Current Conditions Radar Forecasts Rivers and Lakes Climate and Past Weather Local Programs Frost Advisory Sunday Morning Show Caption Click a location below for detailed forecast. Last Map Update: Sun, Feb. 11, 2024 at 5:09:30 am PST Watches, Warnings & Advisories Zoom Out Small Craft Advisory Frost Advisory The National Weather Service forecast for the greater San Francisco Bay Area for Wednesday calls for up to 100% chance of rain during the day, and up to 60% chance of showers with patchy fog in ... San Francisco 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV Meteorologist Kari Hall has a look ahea

/home/none/miniconda3/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


...

/home/none/miniconda3/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


[tool/end] [1:chain:BabyAGI > 63:chain:AgentExecutor > 66:tool:Search > 67:tool:duckduckgo_search] [1.86s] Exiting Tool run with output:
"54 °F like 54° Partly Cloudy N 0 Tomorrow's temperature is forecast to be NEARLY THE SAME as today. Radar Satellite WunderMap |Nexrad Tonight Sun 02/11 Low 44 °F 12% Precip. / 0.00in Partly cloudy... Periods of heavy rain and high elevation snow is expected across the Pacific Northwest through the weekend. Heavy snow and some mixed precipitation is possible through tonight across the Northeast, with more periods of snow this weekend across the region. Read More > San Francisco Bay Area, CA Weather Forecast Office San Francisco, California | Current Weather Forecasts, Live Radar Maps & News | WeatherBug Hourly Today's Weather - San Francisco, CA Feb 06, 2024 1:20 AM SAN FRANCISCO DOWNTOWN CA US NWS -- Feels like -- Hi -- Lo -- -- Live Radar WEATHER DETAILS San Francisco, CA Windchill -- Daily Rain -- Dew Point -- Monthly Rain -- Humidity -- Avg. Wind 

/home/none/miniconda3/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


[tool/end] [1:chain:BabyAGI > 63:chain:AgentExecutor > 70:tool:Search > 71:tool:duckduckgo_search] [1.43s] Exiting Tool run with output:
"SAN FRANCISCO (KGO) -- As the Bay Area recovers from storm impacts, wet weather will continue with a strong storm coming on Sunday, bringing flood watches. Here are the latest updates.... Meanwhile, a dense fog advisory is in effect until 10 a.m. Monday for parts of the North Bay and southern Salinas Valley. The National Weather Service advised of hazardous driving conditions. Updated: Feb 1, 2024 / 06:15 PM PST. SAN FRANCISCO ( KRON) — A powerful storm that bore down on the San Francisco Bay Area on Wednesday continued overnight into Thursday. Officials urged storm ... Wind gusts in San Francisco, Oakland and San Jose are expected to reach 50 to 60 mph Sunday afternoon and evening. Baron/Lynx Wind is expected to switch direction, coming out of the... Follow the latest weather news here or read through the updates below. 5:47 a.m. ET, February 6, 202

{'objective': 'Write a weather report for SF today'}